# Chapter3, Vector Spaces and Subspaces

## 3.1 Spaces of Vectors

이 Chapter는 Vector Space와 Subspace의 정의에 대해 다루기에 구현할 부분이 마땅치 않아 Skip했습니다.

## 3.2 The NullSpace of A : Ax = 0

이 Chapter에서는 Nullspace의 정의와 이를 구하는 방법에 대해 나옵니다.
Nullspace를 구하는 알고리즘을 구현해보겠습니다.
전체적으로는 Elimination 과정과 비슷하며 앞에서 이용했던 Elimination1을 조금 수정했습니다.

In [2]:
import sys

def Find_nullspace(matrix):
    col_of_pivot = 0
    i = 0
    
    while i < len(matrix) and col_of_pivot < len(matrix[0]):
        pivot = matrix[i][col_of_pivot]
        print(f"{i} {col_of_pivot} {pivot}")
        
        if pivot == 0:
            col_of_pivot += 1
        else:
            for j in range(i + 1, len(matrix)):
                multiplier = matrix[j][col_of_pivot] / pivot
                for k in range(col_of_pivot, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i += 1  # 다음 행으로 이동
            col_of_pivot += 1
    
    return matrix
    #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다
    
def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
    
matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()

Find_nullspace(matrix)
print_matrix(matrix)




Matrix A 행 수를 입력하세요: 3
Matrix A 열 수를 입력하세요: 4
행 1을 공백으로 구분하여 입력하세요: 1 1 2 3
행 2을 공백으로 구분하여 입력하세요: 2 2 8 10
행 3을 공백으로 구분하여 입력하세요: 3 3 10 13

0 0 1.0
1 1 0.0
1 2 4.0
2 3 0.0
1.0	1.0	2.0	3.0	
0.0	0.0	4.0	4.0	
0.0	0.0	0.0	0.0	
